In [4]:
import pandas as pd
import numpy as np

data_path = 'Part02/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')
print(df_train.head())
df_train.shape

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


(891, 12)

In [5]:
# 訓練資料需要 train_X, train_Y / 預測輸出需要 ids(識別每個預測值), test_X
# 在此先抽離出 train_Y 與 ids, 而先將 train_X, test_X 該有的資料合併成 df, 先作特徵工程
train_Y = np.log1p(df_train['Survived'])
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'], axis = 1)
df_test = df_test.drop(['PassengerId'], axis = 1)
df = pd.concat([df_train, df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
# 秀出資料欄位的類型, 與對應的數量
# df.dtypes : 轉成以欄位為 index, 類別(type)為 value 的 DataFrame
# .reset_index() : 預設是將原本的 index 轉成一個新的欄位, 如果不須保留 index, 則通常會寫成 .reset_index(drop=True)
dtype_df = df.dtypes.reset_index()
dtype_df.columns = ['Count','Column Type']
dtype_df = dtype_df.groupby('Column Type').aggregate('count').reset_index()
dtype_df

,Column Type,Count
0,int64,3
1,float64,2
2,object,5


In [9]:
# 確定只有 int64, float64, object 三種類型後對欄位名稱執行迴圈, 分別將欄位名稱存於三個 list 中
int_feature = []
float_feature = []
object_feature = []

for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64':
        float_feature.append(feature)
    elif dtype == 'int64':
        int_feature.append(feature)
    else:
        object_feature.append(feature)
print(f'{len(int_feature)} Integer Features: {int_feature}\n')
print(f'{len(float_feature)} Float Feature: {float_feature}\n')
print(f'{len(object_feature)} Object Feature: {object_feature}\n')

3 Integer Features: ['Pclass', 'SibSp', 'Parch']

2 Float Feature: ['Age', 'Fare']

5 Object Feature: ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



# 作業一

In [10]:
df[int_feature].mean()

Pclass    2.294882
SibSp     0.498854
Parch     0.385027
dtype: float64

In [13]:
df[int_feature].max()

Pclass    3
SibSp     8
Parch     9
dtype: int64

In [14]:
# nunique只作用于Series，返回Series中只出现过一次的元素
df[int_feature].nunique()

Pclass    3
SibSp     7
Parch     8
dtype: int64

In [15]:
df[float_feature].mean()

Age     29.881138
Fare    33.295479
dtype: float64

In [16]:
df[float_feature].max()

Age      80.0000
Fare    512.3292
dtype: float64

In [17]:
df[float_feature].nunique()

Age      98
Fare    281
dtype: int64

In [18]:
df[object_feature].mean()
### 非數值型態的資料，因此無法平均

Series([], dtype: float64)

In [19]:
df[object_feature].max()
### 非數值型態的資料，因此無法計算最大值

Name      van Melkebeke, Mr. Philemon
Sex                              male
Ticket                      WE/P 5735
dtype: object

In [20]:
df[object_feature].nunique()
## Sex 是類別資料，無法取得唯一出現的值
## Ticket 每人的編號都不一樣

Name        1307
Sex            2
Ticket       929
Cabin        186
Embarked       3
dtype: int64

# 作業二

#### 試著舉出今天五種類型以外的一種或多種資料類型，你舉出的新類型是否可以歸在三大類中的某些大類?
日期時間的類型，應該歸在object類
#### 所以三大類特徵中，哪一大類處理起來應該最複雜?
object類最麻煩，因為他有多種資料，例如類別、ＩＤ等等，需要再根據其類型做不一樣的操作